# InfluxDB Testing

In [120]:
from influxdb_client import InfluxDBClient
import dotenv
import os

In [121]:
dotenv.load_dotenv('.env')
url = os.getenv("INFLUXDB_URL")
token = os.getenv("INFLUXDB_TOKEN")
org = os.getenv("INFLUXDB_ORG")
bucket = os.getenv("INFLUXDB_BUCKET")

In [209]:
query = f'from(bucket:"{bucket}") |> range(start: -12m, stop: -2m) \
  |> filter(fn: (r) => r["entity_id"] == "sonoff_1001e01c1e_power")\
  |> filter(fn: (r) => r["_field"] == "value")\
  '

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()
tables = query_api.query(query=query)

In [210]:
measurement = []
listoflist = []
i = 0
for table in tables:
    for row in table.records:
        record = [row.values["_time"], row.values["_value"]]
        listoflist.append(record)
        print(f'{row.values["_time"]}, {row.values["_value"]}') # row.values is a dictionary
        value = row.values["_value"]
        measurement.append(value)

2024-02-09 07:22:43.261147+00:00, 22.24
2024-02-09 07:23:13.262131+00:00, 24.56
2024-02-09 07:23:25.263276+00:00, 22.92
2024-02-09 07:23:29.260683+00:00, 25.48
2024-02-09 07:23:31.262393+00:00, 23.63
2024-02-09 07:23:43.259983+00:00, 23.96
2024-02-09 07:23:49.261148+00:00, 42.79
2024-02-09 07:23:51.261610+00:00, 27.13
2024-02-09 07:23:53.261756+00:00, 22.44
2024-02-09 07:23:55.262296+00:00, 24.44
2024-02-09 07:24:01.262302+00:00, 24.16
2024-02-09 07:24:03.261539+00:00, 22.02
2024-02-09 07:24:05.261849+00:00, 24.3
2024-02-09 07:24:07.262457+00:00, 21.81
2024-02-09 07:24:11.261967+00:00, 23.86
2024-02-09 07:24:13.262135+00:00, 22.01
2024-02-09 07:24:19.262297+00:00, 21.97
2024-02-09 07:24:21.262503+00:00, 25.38
2024-02-09 07:24:33.263394+00:00, 23.11
2024-02-09 07:24:37.262366+00:00, 24.15
2024-02-09 07:24:43.275536+00:00, 23.83
2024-02-09 07:24:47.262380+00:00, 26.32
2024-02-09 07:24:49.261615+00:00, 22.47
2024-02-09 07:25:01.264430+00:00, 24.68
2024-02-09 07:25:05.262936+00:00, 23.3
20

In [211]:
t_initial = listoflist[0][0].timestamp()
for i in range(len(listoflist)):
    listoflist[i][0] = listoflist[i][0].timestamp() - t_initial

In [212]:
area = 0
for i in range(len(listoflist)-1):
    ts_i = listoflist[i][0]
    ts_f = listoflist[i+1][0]
    v_i = listoflist[i][1]
    v_f = listoflist[i+1][1]
    elapsed = ts_f - ts_i
    area += (v_i * elapsed + (v_f - v_i) * elapsed / 2) / listoflist[-1][0] # trapezoidal rule
area

24.094547813520286

In [213]:
sum(measurement) / len(measurement)

24.400952380952383

Generating Random Data for Analysis

In [111]:
import random

random_int_c7 = [random.randint(2200,3000)/100 for i in range(200)]
random_int_c3 = [random.randint(2250,3000)/100 for i in range(200)]
random_int_c0 = [random.randint(2300,3000)/100 for i in range(200)]

In [112]:
def write_random(c, rand_list, writer):
    hrs = 9
    for i in rand_list:
        hrs += 1
        writer.write(f'Timestamp, {hrs}, {i}, {c}\n')


with open('influx-test.csv','w') as csvfile:
    csvfile.write('Timestamp, Hrs, Value, C-State\n')
    write_random('C7', random_int_c7, csvfile)
    write_random('C3', random_int_c3, csvfile)
    write_random('C0', random_int_c0, csvfile)
